# APK 4720 / APK 6725

# Assignment # 5

Please submit your assignment as a Jupyter  notebook (.ipynb file). Start a new Jupyter notebook and name it "YourName_Assignment_5"

Replace *YourName* with your first name and last name

## Background

This assignment is not directly related to human movement; however, I decided to use this data, as it will allow us to explore important concepts related to correlated input variables and the importance of scaling your input data for model interpretation. 


Lumbar lordosis describes the curvature of the lower spine and is strongly related to pelvic geometry. In this assignment, you will build linear regression models to predict lumbar lordosis angle from pelvic biomechanical measurements.

The goal is to understand:
- How feature relationships influence model design.
- Scaling and why it matters.
- How to evaluate models.

## Data
The CSV file `assignment_4.csv` contains the following input columns:

- `pelvic_incidence`
- `pelvic_tilt`
- `sacral_slope`
- `pelvic_radius`

and output column:

- `lumbar_lordosis_angle`

All variables are continuous biomechanical measurements.

## Part A (1 point)

1. Load the dataset into a pandas DataFrame.
2. Display the first few rows of the dataset
3. Report:
    - number of samples
    - number of variables
4. Remove all rows containing missing values
5. Define two NumPy arrays:
   - `y` -> The target variable, corresponding to the `lumbar_lordosis_angle` column
   - `X` -> The rest of the columns in the dataset

## Part B (1 points) 
1. Split the data into train and validation sets
   - 75% of the data for training
   - 25% of the data for testing
Use a fixed random state for reproducibility `random_state = 10`
2. Train a linear regression model using all the input features
3. Evaluate the model performance on the test set using 
   - $R^2$ score
   - Adjusted $R^2$ score
4. What can you say about the resulting model? 

Note: the adjusted $R^2$ score can be computed as follows:
```Python
def calculate_adjusted_r2(r2_score, n_observations, n_predictors):
    adj_r2 = 1 - (1 - r2_score) * ((n_observations - 1) / (n_observations - n_predictors - 1))
    return adj_r2


r2score_test = model.score(X_test,y_test)
adj_r2score_test = calculate_adjusted_r2(r2score_test, X_test.shape[0], X_test.shape[1])
```

## Part C (0.5 points)
1. Compute the linear correlation of the input features using the following code
```Python
corr = X.corr() #X must be a Pandas Dataframe
print(corr)
```
2. Identify which variables are strongly correlated (>0.8) and highly correlated (>0.6).
3. Do these correlations make sense? 

## Part D (2 points)
Highly correlated inputs might result in estimation errors, resulting in poor generalizability. It is often a good practice to reduce the linearly correlated inputs.

In this part of the assignment, you will use cross-validation to compare two models:
1. A model that uses all the available inputs
2. A model that uses only non-correlated inputs

For this, you will train a model by removing features that are highly correlated with others. I suggest removing one feature at a time and training/evaluating the model with each feature set. 
For each new set of features, you must report the cross-validation average $R^2$ score. 

3. Train a final model using the feature set that provided the highest average cross-validation $R^2$ score.
4. Evaluate the model performance on the test set using 
   - $R^2$ score
   - Adjusted $R^2$ score
5. What can you say about the resulting model? How is the model compared to the one that you found in part B? 

Note: You can use `scikit-learn` pre-built functions to perform these tasks. The following code allows you to compute the average cross-validation $R^2$ score using the training data. 

```Python
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
        LinearRegression(fit_intercept=True),
        X_train[['pelvic_incidence', 'pelvic_tilt', 'sacral_slope', 'pelvic_radius']],
        y_train,
        cv=5,
        scoring="r2"
    )
print(scores.mean(), scores.std())
```


## Part E (0.5 points)

After completing the previous section, you should have ended up with a model that has only two features:
- `pelvic_incidence`
- `pelvic_radius`

You might be interested in figuring out how much each of these variables influences the model prediction. For this, you need to look at the model coefficients and figure out which one is larger (in absolute units). However, each one of these variables has its own scale; pelvic incidence is measured in degrees and pelvic radius in millimeters, so that the model coefficients cannot be compared directly. 

To facilitate comparison between the different model coefficients, it is necessary to scale the variables so they are all of the same scale. A common approach is to remove the mean and divide by the standard deviation, so that all the variables have a zero mean and a unit standard deviation. In this way, all variables have comparable ranges. While ordinary linear regression does not require scaling to make predictions, scaling:

- improves coefficient interpretability,
- ensures fair comparison across features,

The following code allows you to train a model with `pelvic_incidence` and `pelvic_radius` as input variables after scaling the input variables so they all have a zero mean and a unit standard deviation. 
```Python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

selected_features = ['pelvic_incidence', 'pelvic_radius']
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LinearRegression(fit_intercept=True))
])

pipe.fit(X_train[selected_features], y_train)
```

You can obtain the model coefficients as follows :
```Python
print(pipe.named_steps['lr'].coef_)
```

1. Train a model with a standard scaler and another without it using  `['pelvic_incidence', 'pelvic_radius']` as model coefficients and using the training data.
2. Compare the magnitude of the coefficients for the models with and without scaling.
3. Why is comparing the coefficients before scaling misleading?
4. Remove the feature with the smallest coefficient 


In [77]:
import pandas as pd
import numpy as np

df = pd.read_csv("assignment_4.csv")
target = "lumbar_lordosis_angle"
X = df.drop(columns=[target])
y = df[target]

In [8]:
corr = X.corr() #X must be a Pandas Dataframe
print(corr)

                  pelvic_incidence  pelvic_tilt  sacral_slope  pelvic_radius
pelvic_incidence          1.000000     0.629199      0.814960      -0.247467
pelvic_tilt               0.629199     1.000000      0.062345       0.032668
sacral_slope              0.814960     0.062345      1.000000      -0.342128
pelvic_radius            -0.247467     0.032668     -0.342128       1.000000


In [5]:
corr

,pelvic_incidence,pelvic_tilt,sacral_slope,pelvic_radius
pelvic_incidence,1.000000,0.629199,0.814960,-0.247467
pelvic_tilt,0.629199,1.000000,0.062345,0.032668
sacral_slope,0.814960,0.062345,1.000000,-0.342128
pelvic_radius,-0.247467,0.032668,-0.342128,1.000000


In [7]:
df.corr()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius
pelvic_incidence,1.000000,0.629199,0.717282,0.814960,-0.247467
pelvic_tilt,0.629199,1.000000,0.432764,0.062345,0.032668
lumbar_lordosis_angle,0.717282,0.432764,1.000000,0.598387,-0.080344
sacral_slope,0.814960,0.062345,0.598387,1.000000,-0.342128
pelvic_radius,-0.247467,0.032668,-0.080344,-0.342128,1.000000


In [86]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

model = LinearRegression(fit_intercept=True)
model.fit(X_train,y_train)

def calculate_adjusted_r2(r2_score, n_observations, n_predictors):
    adj_r2 = 1 - (1 - r2_score) * ((n_observations - 1) / (n_observations - n_predictors - 1))
    return adj_r2


r2score_test = model.score(X_test,y_test)
adj_r2score_test = calculate_adjusted_r2(r2score_test, X_test.shape[0], X_test.shape[1])

print(r2score_test, adj_r2score_test )

0.6954055410030459 0.6740304912488737


In [72]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LinearRegression(fit_intercept=True))
])

pipe.fit(X_train, y_train)
r2_score_scaled_test = pipe.score(X_test,y_test)
adj_r2score_scaled_test = calculate_adjusted_r2(r2score_test, X_test.shape[0], X_test.shape[1])
print("Scaled Test R2:", r2_score_scaled_test)
print("Scaled Test MSE:", adj_r2score_scaled_test)

Scaled Test R2: 0.6954055616424442
Scaled Test MSE: 0.6673519331953717


In [65]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
        LinearRegression(fit_intercept=True),
        X_train[['pelvic_incidence', 'pelvic_radius']],
        y_train,
        cv=5,
        scoring="r2"
    )
print(scores.mean(), scores.std())

0.4451084840943677 0.10558040841571127


In [17]:
scores.

array([ 1.13341531e+07, -1.13341524e+07, -1.13341523e+07,  1.33995172e-01])

In [58]:
scores.partition(1)

In [96]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train[['pelvic_incidence']],y_train)
r2score_test = model.score(X_test[['pelvic_incidence']],y_test)
print(r2score_test, adj_r2score_test )

0.6711622692705193 0.6656816404250279


In [75]:
X[['pelvic_incidence', 'pelvic_radius']].mean()

pelvic_incidence     60.496653
pelvic_radius       117.920655
dtype: float64

In [74]:
X[['pelvic_incidence', 'pelvic_radius']].min()

pelvic_incidence    26.147921
pelvic_radius       70.082575
dtype: float64

In [97]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

selected_features = ['pelvic_incidence',  'pelvic_radius']#['pelvic_incidence', 'pelvic_radius']
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LinearRegression(fit_intercept=True))
])

pipe.fit(X_train[selected_features], y_train)

,steps,"[('scaler', ...), ('lr', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None


In [98]:
pipe.named_steps['lr'].coef_

array([13.04278984,  1.69734801])